In [2]:
from collections import defaultdict, Counter
import json
import re
import os

from common import *

from matplotlib import pyplot as plt
import pandas as pd

In [3]:
# File generated with
# python analyze_bundle_dataset.py --total 100000 --worker $(nproc) -s get_urls $DATASETS/bundles-daily/results-*
# mv results.json bundlers.json
with open(os.path.join(DATASETS, "cdn-results.json"), "r") as f:
    data = json.load(f)

In [ ]:
responsive_data = [d["urls"] for d in data if len(d["urls"]) > 0]

In [11]:
results = defaultdict(dict)

In [12]:
for host in CDN_HOSTS:
    count = 0
    for urls in responsive_data:
        for url in urls:
            if host in url:
                count += 1
    results["occurences"][host] = count

In [13]:
for host in CDN_HOSTS:
    count = 0
    for urls in responsive_data:
        for url in urls:
            if host in url:
                count += 1
                break
    results["at_least_one"][host] = count

In [14]:
version_regex = re.compile(r"[0-9]+\.[0-9]+\.[0-9]+")
no_match = []
for host in CDN_HOSTS:
    count = 0
    for urls in responsive_data:
        for url in urls:
            if host in url:
                if version_regex.search(url):
                    count += 1
                else:
                    no_match.append(url)
    results["with_versions"][host] = count

In [15]:
results

defaultdict(dict,
            {'occurences': {'//cdn.jsdelivr.net': 373251,
              '//cdnjs.cloudflare.com': 497213,
              '//unpkg.com': 85646,
              '//ajax.googleapis.com': 278897,
              '//ajax.aspnetcdn.com': 9391,
              '//code.jquery.com': 157443},
             'at_least_one': {'//cdn.jsdelivr.net': 232982,
              '//cdnjs.cloudflare.com': 259470,
              '//unpkg.com': 54355,
              '//ajax.googleapis.com': 242591,
              '//ajax.aspnetcdn.com': 5966,
              '//code.jquery.com': 135218},
             'with_versions': {'//cdn.jsdelivr.net': 232631,
              '//cdnjs.cloudflare.com': 482685,
              '//unpkg.com': 14395,
              '//ajax.googleapis.com': 264956,
              '//ajax.aspnetcdn.com': 6891,
              '//code.jquery.com': 153804}})

### How old are the jQuery versions?

In [ ]:
version_regex = re.compile(r"([0-9]+\.[0-9]+\.[0-9]+)\.[^?]+$")
host = CDN_HOSTS[-1]
versions = Counter()
versions_major = Counter()
for urls in responsive_data:
    for url in urls:
        if host in url:
            matches = version_regex.search(url)
            if matches:
                versions.update([matches.groups()[0]])
                versions_major.update([matches.groups()[0][0]])

In [32]:
(versions_major["1"] + versions_major["2"]) / (sum(versions_major.values()))

0.2427144956479144

### What about cookielaw?

In [6]:
cookielaw_stats = {}
for d in data:
    if len(d["urls"]) > 0:
        cookielaw = cookielaw_stats.get(d["domain"], False)
        for url in d["urls"]:
            if "cdn.cookielaw.org" in url:
                cookielaw = True
                break
        cookielaw_stats[d["domain"]] = cookielaw
print(sum(map(int, cookielaw_stats.values())), len(cookielaw_stats))

3611 78394


In [7]:
3611 / 78394

0.04606219863765084